In [65]:
import json
from pathlib import Path
import gurobipy as gp
from gurobipy import GRB


In [66]:
# Function to load JSON data
def load_json(path: Path):
    with open(path, "r") as f:
        return json.load(f)

In [67]:
def run_flex_load_pv_optimization(bus_params, der_production, usage_preference, appliance_params):
    
    # Define model parameters
    T = len(der_production[0]["hourly_profile_ratio"])
    pv_max = der_production[0]["hourly_profile_ratio"]
    min_daily_energy = usage_preference[0]["load_preferences"][0]["min_total_energy_per_day_hour_equivalent"]
    max_import = bus_params[0]["max_import_kW"]
    max_export = bus_params[0]["max_export_kW"]
    import_tariff = bus_params[0]["import_tariff_DKK/kWh"]
    export_tariff = bus_params[0]["export_tariff_DKK/kWh"]
    energy_price = bus_params[0]["energy_price_DKK_per_kWh"]

    max_load = appliance_params["load"][0]["max_load_kWh_per_hour"]
    PVcap = appliance_params["DER"][0]["max_power_kW"]
    
    # Create optimization model
    m = gp.Model("flexible_load_pv")


    # Decision variables
    PD = {t: m.addVar(lb=0, ub=max_load, name=f"PD_{t}") for t in range(T)}
    PG = {t: m.addVar(lb=0, ub=pv_max[t]*PVcap, name=f"PG_{t}") for t in range(T)}
    PIMP = {t: m.addVar(lb=0, ub=max_import, name=f"PIMP_{t}") for t in range(T)}
    PEXP = {t: m.addVar(lb=0, ub=max_export, name=f"PEXP_{t}") for t in range(T)}
    PV_curtail = {t: m.addVar(lb=0, ub=PVcap, name=f"PV_curtail_{t}") for t in range(T)}



    # Constraints
    for t in range(T):
        m.addConstr(PIMP[t] - PEXP[t] == PD[t] - PG[t], name=f"balance_{t}")
        m.addConstr(PG[t] + PV_curtail[t] == PVcap, name=f"pv_cap_{t}")

    m.addConstr(gp.quicksum(PD[t] for t in range(T)) >= min_daily_energy, name="daily_min_energy")


    # Full objective function
    obj = gp.quicksum(
        energy_price[t] * (PIMP[t] - PEXP[t]) +
        import_tariff * PIMP[t] +
        export_tariff * PEXP[t] for t in range(T)
    )
    
    # Set objective 
    m.setObjective(obj, GRB.MINIMIZE)

    # Optimize model
    m.optimize()
    
    # Check if optimization is successful
    if m.Status != GRB.OPTIMAL:
        raise RuntimeError(f"Solver ended with status {m.Status}")


    # Extract results
    results = []
    for t in range(T):
        imp = PIMP[t].X
        exp = PEXP[t].X
        pv_used = PG[t].X
        demand = PD[t].X
        net_grid = imp - exp
        pv_curt = PV_curtail[t].X
        results.append([t, imp, exp, pv_used, demand, net_grid, pv_curt])

    return results, m.ObjVal


In [68]:
def main():
    # File paths
    base_dir = Path().resolve().parent / "data" / "question_1a"

    # Load input data
    appliance_params = load_json(base_dir / "appliance_params.json")
    bus_params = load_json(base_dir / "bus_params.json")
    der_production = load_json(base_dir / "DER_production.json")
    usage_preference = load_json(base_dir / "usage_preference.json")
    
    # Run optimization
    results, obj_val = run_flex_load_pv_optimization(bus_params, der_production, usage_preference, appliance_params)


    #Change in values for a5
    

    # Change export tariff to 100 DKK/kWh, so that export is not an option
    bus_params_a5 = bus_params.copy()
    bus_params_a5[0]["export_tariff_DKK/kWh"] = 100
    
    resultsa5, obj_vala5 = run_flex_load_pv_optimization(bus_params_a5, der_production, usage_preference, appliance_params)
    
    # Change electricity prices to negative values

    
    
    # Print results
    #print(f"Objective (total cost): {obj_val:.2f} DKK\n")
    #print("Hour | Import | Export | PV_used | Demand | NetGrid | PV_Curtail")
    #print("-" * 70)
    #for row in results:
    #    print(f"{row[0]:>4} | {row[1]:>6.2f} | {row[2]:>6.2f} | {row[3]:>7.2f} | {row[4]:>6.2f} | {row[5]:>7.2f} | {row[6]:>9.2f}")
    #print("Total import:",sum(results[row][1] for row in range(len(results))))
    #print("Total consumption:", round(sum(results[row][4] for row in range(len(results)))))
    #
    ## Print for a5
    #print(f"Objective (total cost): {obj_vala5:.2f} DKK\n")
    #print("Hour | Import | Export | PV_used | Demand | NetGrid | PV_Curtail")
    #print("-" * 70)
    #for row in resultsa5:
    #    print(f"{row[0]:>4} | {row[1]:>6.2f} | {row[2]:>6.2f} | {row[3]:>7.2f} | {row[4]:>6.2f} | {row[5]:>7.2f} | {row[6]:>9.2f}")
    #print("Total import:",sum(resultsa5[row][1] for row in range(len(resultsa5))))
    #print("Total consumption:", round(sum(resultsa5[row][4] for row in range(len(resultsa5)))))
    
    # Print differences
    print("\nDifferences between original and a5 scenario:")
    print("Hour | Import Diff | Export Diff | PV_used Diff | Demand Diff | NetGrid Diff | PV_Curtail Diff")
    print("-" * 80)
    for row in range(len(results)):
        imp_diff = resultsa5[row][1] - results[row][1]
        exp_diff = resultsa5[row][2] - results[row][2]
        pv_used_diff = resultsa5[row][3] - results[row][3]
        demand_diff = resultsa5[row][4] - results[row][4]
        netgrid_diff = resultsa5[row][5] - results[row][5]
        pv_curtail_diff = resultsa5[row][6] - results[row][6]
        print(f"{row:>4} | {imp_diff:>11.2f} | {exp_diff:>11.2f} | {pv_used_diff:>13.2f} | {demand_diff:>11.2f} | {netgrid_diff:>12.2f} | {pv_curtail_diff:>15.2f}")
    
    
if __name__ == "__main__":
    main()
    
    
    
    

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 5 6600U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 49 rows, 120 columns and 168 nonzeros
Model fingerprint: 0xc42e98fb
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 3e+00]
  Bounds range     [2e-01, 1e+03]
  RHS range        [3e+00, 8e+00]
Presolve removed 33 rows and 66 columns
Presolve time: 0.02s
Presolved: 16 rows, 54 columns, 69 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -7.0800000e+03   1.498106e+04   0.000000e+00      0s
      13   -6.3725000e+00   0.000000e+00   0.000000e+00      0s

Solved in 13 iterations and 0.03 seconds (0.00 work units)
Optimal objective -6.372500000e+00
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 5 6600U 

In [69]:
def run_flex_load_pv_optimization_w_battery(bus_params, der_production, usage_preference, appliance_params):
    
    # Define model parameters
    T = len(der_production[0]["hourly_profile_ratio"])
    pv_max = der_production[0]["hourly_profile_ratio"]
    min_daily_energy = usage_preference[0]["load_preferences"][0]["min_total_energy_per_day_hour_equivalent"]
    max_import = bus_params[0]["max_import_kW"]
    max_export = bus_params[0]["max_export_kW"]
    import_tariff = bus_params[0]["import_tariff_DKK/kWh"]
    export_tariff = bus_params[0]["export_tariff_DKK/kWh"]
    energy_price = bus_params[0]["energy_price_DKK_per_kWh"]
    
    charging_power = appliance_params["storage"][0]["max_charging_power_ratio"]
    discharging_power = appliance_params["storage"][0]["max_discharging_power_ratio"]
    efficiency_charge = appliance_params["storage"][0]["charging_efficiency"]
    efficiency_discharge = appliance_params["storage"][0]["discharging_efficiency"]

    max_load = appliance_params["load"][0]["max_load_kWh_per_hour"]
    PVcap = appliance_params["DER"][0]["max_power_kW"]
    battery_capacity = appliance_params["storage"][0]["storage_capacity_kWh"]
    
    #Calculations
    actual_charging = charging_power * battery_capacity
    actual_discharging = discharging_power * battery_capacity
    
    
    # Create optimization model
    m = gp.Model("flexible_load_pv")


    # Decision variables
    PD = {t: m.addVar(lb=0, ub=max_load, name=f"PD_{t}") for t in range(T)}
    PG = {t: m.addVar(lb=0, ub=pv_max[t]*PVcap, name=f"PG_{t}") for t in range(T)}
    PIMP = {t: m.addVar(lb=0, ub=max_import, name=f"PIMP_{t}") for t in range(T)}
    PEXP = {t: m.addVar(lb=0, ub=max_export, name=f"PEXP_{t}") for t in range(T)}
    PV_curtail = {t: m.addVar(lb=0, ub=PVcap, name=f"PV_curtail_{t}") for t in range(T)}
    
    #Battery decision variables
    P_charge = {t: m.addVar(lb=0, ub=actual_charging, name=f"charging_power_{t}") for t in range(T)}
    P_discharge = {t: m.addVar(lb=0, ub=actual_discharging, name=f"discharging_power_{t}") for t in range(T)}
    SOC = {t: m.addVar(lb=0, ub= battery_capacity, name=f"state_of_charge_{t}") for t in range(T)}

    # Battery start and stop
    m.addConstr(SOC[0]==battery_capacity * 0.5)
    m.addConstr(SOC[23] == SOC[0])

    # Constraints
    for t in range(T):
        m.addConstr(PIMP[t] - PEXP[t] == PD[t] - PG[t] + P_charge[t] - P_discharge[t], name=f"balance_w_battery_{t}")
        m.addConstr(PG[t] + PV_curtail[t] == pv_max[t] * PVcap, name=f"pv_cap_{t}")
        
    #Ramping constraints
    for t in range(0, T-1):
        m.addConstr(SOC[t+1] == SOC[t] + (P_charge[t] * efficiency_charge) - (P_discharge[t] / efficiency_discharge), name=f"battery_content{t}")



    # Only for 1a   
    #m.addConstr(gp.quicksum(PD[t] for t in range(T)) >= min_daily_energy, name="daily_min_energy")


    # Full objective function
    obj = gp.quicksum(
        energy_price[t] * (PIMP[t] - PEXP[t]) +
        import_tariff * PIMP[t] +
        export_tariff * PEXP[t] for t in range(T)
    )
    
    # Set objective 
    m.setObjective(obj, GRB.MINIMIZE)

    # Optimize model
    m.optimize()
    
    # Check if optimization is successful
    if m.Status != GRB.OPTIMAL:
        raise RuntimeError(f"Solver ended with status {m.Status}")


    # Extract results
    results = []
    for t in range(T):
        imp = PIMP[t].X
        exp = PEXP[t].X
        pv_used = PG[t].X
        demand = PD[t].X
        net_grid = imp - exp
        pv_curt = PV_curtail[t].X
        results.append([t, imp, exp, pv_used, demand, net_grid, pv_curt])

    return results, m.ObjVal


In [70]:
def main():
    # File paths
    base_dir = Path().resolve().parent / "data" / "question_1c"

    # Load input data
    appliance_params = load_json(base_dir / "appliance_params.json")
    bus_params = load_json(base_dir / "bus_params.json")
    der_production = load_json(base_dir / "DER_production.json")
    usage_preference = load_json(base_dir / "usage_preferences.json")
    
    # Run optimization
    results_battery, obj_val_battery = run_flex_load_pv_optimization_w_battery(bus_params, der_production, usage_preference, appliance_params)

if __name__ == "__main__":
    main()


Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 5 6600U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 73 rows, 192 columns and 287 nonzeros
Model fingerprint: 0x8c12abf3
Coefficient statistics:
  Matrix range     [9e-01, 1e+00]
  Objective range  [4e-01, 3e+00]
  Bounds range     [2e-01, 1e+03]
  RHS range        [2e-01, 3e+00]
Presolve removed 29 rows and 84 columns
Presolve time: 0.01s
Presolved: 44 rows, 108 columns, 174 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -9.4404604e+03   9.028219e+03   0.000000e+00      0s
      65   -1.6728840e+01   0.000000e+00   0.000000e+00      0s

Solved in 65 iterations and 0.02 seconds (0.00 work units)
Optimal objective -1.672884000e+01
